## 开始使用 Keras Sequential 顺序模型

顺序模型是多个网络层的线性堆叠。

你可以通过将网络层实例的列表传递给 `Sequential` 的构造器，来创建一个 `Sequential` 模型：

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

model = keras.models.Sequential([
    keras.layers.Dense(32, input_shape=(784,)),
    keras.layers.Activation(tf.nn.relu),
    keras.layers.Dense(10),
    keras.layers.Activation(tf.nn.softmax)
])

也可以简单的使用 `add()` 方法将各层添加到模型中：

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(32, input_shape=(784,)))
model.add(keras.layers.Activation(tf.nn.relu))

----
#### 指定输入数据的尺寸

模型需要知道它所期望的输入的尺寸。
出于这个原因，顺序模型中的第一层（且只有第一层，因为下面的层可以自动地推断尺寸）需要接收关于其输入尺寸的信息。
有几种方法来做到这一点：

* 传递一个 `input_shape` 参数给第一层。它是一个表示尺寸的元组 (一个由整数或 `None` 组成的元组，其中 `None` 表示可能为任何正整数)。在 `input_shape` 中不包含数据的 `batch` 大小。
* 某些 2D 层，例如 `Dense`，支持通过参数 `input_dim` 指定输入尺寸，某些 3D 时序层支持 `input_dim` 和 `input_length` 参数。
* 如果你需要为你的输入指定一个固定的 `batch` 大小（这对 stateful RNNs 很有用），你可以传递一个 `batch_size` 参数给一个层。如果你同时将 `batch_size=32` 和 `input_shape=(6, 8)` 传递给一个层，那么每一批输入的尺寸就为 `(32，6，8)`。

因此，下面的代码片段是等价的

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(32, input_shape=(784,))
])

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(32, input_dim=784)
])

----
#### 模型编译
在训练模型之前，您需要配置学习过程，这是通过 `compile` 方法完成的。它接收三个参数：
* 优化器 `optimizer`。它可以是现有优化器的字符串标识符，如 `rmsprop` 或 `adagrad`，也可以是 `Optimizer` 类的实例。详见：[optimizers](https://keras-zh.readthedocs.io/optimizers/)
* 损失函数 `loss`，模型试图最小化的目标函数。它可以是现有损失函数的字符串标识符，如 `categorical_crossentropy` 或 `mse`，也可以是一个目标函数。详见：[losses](https://keras-zh.readthedocs.io/losses)。
* 评估标准 `metrics`。对于任何分类问题，你都希望将其设置为 `metrics = ['accuracy']`。评估标准可以是现有的标准的字符串标识符，也可以是自定义的评估标准函数。详见: [metrics](https://keras-zh.readthedocs.io/metrics)。

In [ ]:
# 多分类问题
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.categorical_crossentropy,
    metrics=[keras.metrics.Accuracy()]
)

# 二分类问题
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.Accuracy()]
)

# 均方误差回归问题
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.mse
)

# 自定义评估标准函数
import tensorflow.keras.backend as K


def mean_pred(y_true, y_pred):
    return K.mean(y_pred)


model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.binary_crossentropy,
              metrics=[keras.metrics.Accuracy(), mean_pred])

----
## 模型训练
`Keras` 模型在输入数据和标签的 `Numpy` 矩阵上进行训练。为了训练一个模型，你通常会使用 `fit` 函数。
[文档详见此处](https://keras-zh.readthedocs.io/models/sequential)。

In [ ]:
# 对于只有 2 个类的但输入模型（二进制分类）：

model = keras.models.Sequential([
    keras.layers.Dense(32, activation=keras.activations.relu, input_dim=100),
    keras.layers.Dense(1, activation=keras.activations.sigmoid)
])
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.binary_crossentropy,
              metrics=[keras.metrics.Accuracy()])

# 生成虚拟数据
import numpy as np

data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))
# 训练模型，以 32 个样本为一个 batch 进行迭代
model.fit(data, labels, epochs=10, batch_size=32)

In [ ]:
# 对于具有 10 个类的单输入模型（多分类分类）：

model = keras.models.Sequential([
    keras.layers.Dense(32, activation=keras.activations.relu, input_dim=100),
    keras.layers.Dense(10, activation=keras.activations.softmax)
])
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.Accuracy()])

# 生成虚拟数据
import numpy as np

data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# 将标签转换为分类的 one-hot 编码
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

# 训练模型，以 32 个样本为一个 batch 进行迭代
model.fit(data, one_hot_labels, epochs=10, batch_size=32)